In [4]:
import bagpy
from bagpy import bagreader
import numpy as np
import pandas as pd

# Read the bag file
bag = bagreader(r"/media/kevin/X9 Pro/484_final_project/rosbags/2024-11-29-16-45-01-kiss.bag")

# Get list of topics
print(bag.topic_table)

# Read point cloud messages
pointcloud_msgs = bag.message_by_topic('/synchronized_cloud_with_pose')  # adjust topic name as needed
df_pointcloud = pd.read_csv(pointcloud_msgs)

[INFO]  Data folder /media/kevin/X9 Pro/484_final_project/rosbags/2024-11-29-16-45-01-kiss already exists. Not creating.
                           Topics                                     Types  \
0                    /diagnostics           diagnostic_msgs/DiagnosticArray   
1                   /joint_states                    sensor_msgs/JointState   
2                     /kiss/frame                   sensor_msgs/PointCloud2   
3                 /kiss/keypoints                   sensor_msgs/PointCloud2   
4                 /kiss/local_map                   sensor_msgs/PointCloud2   
..                            ...                                       ...   
71    /septentrio_gnss/twist_gnss  geometry_msgs/TwistWithCovarianceStamped   
72     /septentrio_gnss/twist_ins  geometry_msgs/TwistWithCovarianceStamped   
73  /synchronized_cloud_with_pose                   sensor_msgs/PointCloud2   
74                            /tf                        tf2_msgs/TFMessage   
75        

In [2]:
!pip3 install bagpy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 11.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for gnupg: filename=gnupg-2.3.1-py3-none-any.whl size=94617 sha256=85b50d86509683930a8faf806ac904faf5030ee394e03568c574f3aa3efb61c8
  Stored in directory: /home/kevin/.cache/pip/wheels/26/3f/50/9d97272d89c993d4c347b425fd105fc899a447cb104ac4d63c
Successfully built gnupg
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.1.4
    Uninstalling Jinja2-3.1.4:
      Successfully uninstalled Jinja2-3.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
triton 2.0.0 requires cmake, which is not installed.
triton 2.0.0 requires lit, which is not ins

In [9]:
def parse_ouster_frame(row):
    # Get binary data
    binary_data = eval(row['data'])  # Convert string representation to bytes
    
    # Get dimensions
    width = row['width']
    height = row['height']
    point_step = row['point_step']
    
    # Parse to float32 array
    points = np.frombuffer(binary_data, dtype=np.float32)
    
    
    # Each point has 12 float32 values (48 bytes / 4 bytes per float32 = 12)
    # Typically: x, y, z, intensity, t, reflectivity, ring, noise, range
    points = points.reshape(-1, 4)
    
    # Get just x, y, z, intensity
    xyz_intensity = points
    
    # Reshape to original lidar structure if needed
    # xyz_intensity = xyz_intensity.reshape(height, width, 12)
    
    return xyz_intensity

In [6]:
print(df_pointcloud)

             Time  header.seq  header.stamp.secs  header.stamp.nsecs  \
0    1.732920e+09         194         1732920299           895498407   
1    1.732920e+09         195         1732920300           195521450   
2    1.732920e+09         196         1732920300           495606954   
3    1.732920e+09         197         1732920300           795497254   
4    1.732920e+09         198         1732920301            95521054   
..            ...         ...                ...                 ...   
128  1.732920e+09         322         1732920360           195527480   
129  1.732920e+09         323         1732920360           695483055   
130  1.732920e+09         324         1732920360           995531351   
131  1.732920e+09         325         1732920361           295636624   
132  1.732920e+09         326         1732920361           495551590   

    header.frame_id  height  width  \
0         os_sensor       1  49152   
1         os_sensor       1  49152   
2         os_sensor  

In [10]:
# Process all frames
frames = []
timestamps = []

for idx, row in df_pointcloud.iterrows():
    frame = parse_ouster_frame(row)
    frames.append(frame)
    timestamps.append(row['Time'])

# Convert to numpy array
point_cloud_array = np.array(frames)
timestamps = np.array(timestamps)

In [15]:
point_cloud_array.shape
first_frame = point_cloud_array[0]
print(first_frame.shape)
x = first_frame[:, 0]  # x coordinates
y = first_frame[:, 1]  # y coordinates
z = first_frame[:, 2]  # z coordinates
intensity = first_frame[:, 3]  # intensity values
print(x)
print(y)
print(z)
print(intensity)

(49152, 4)
[-24.558678  -24.566994  -24.563416  ...  -2.2909186  -2.297761
  -2.2998097]
[-4.7811394 -4.6024756 -4.4219904 ... -0.6379234 -0.6003195 -0.5617764]
[-2.960994  -2.9584274 -2.9546835 ... -2.319991  -2.3196476 -2.3174653]
[ 80.  80.  80. ... 127. 127. 127.]


In [16]:
point_cloud_array.shape

(133, 49152, 4)

In [18]:
flattened_array = point_cloud_array.reshape(133, 49152, 4)
print(flattened_array.shape)
np.save('all_points.npy', flattened_array)

(133, 49152, 4)
